In [3]:
# -*- coding:utf-8 -*-

import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.4f' % x) # 不使用科学计数法
pd.set_option('mode.chained_assignment', None) # 关闭 SettingWithCopyWarning
pd.set_option('display.max_columns', None) # 显示所有列
pd.set_option('display.max_rows', 10) # 设定df全部显示的行数

# 一个cell做多个输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

## 层次化索引
> 尽量避免构建层次化行索引的df，因为去内层索引不好操作，但是遇见也应知道该如何操作（reset_index,unstack）；该部分尽管不常用，但非常重要，具体参考[pandas MultiIndex 1](https://blog.csdn.net/PIPIXIU/article/details/80232805) 和 [pandas MultiIndex 2](https://www.cnblogs.com/dataAnalysis/p/9329827.html)

在一个轴上拥有多个（两个以上）索引级别。抽象点说，它使你能以低维度形式处理高维度数据；

- 对于一个层次化索引的对象，可以使用所谓的部分索引，使用它选取数据子集操作更简单
    > 针对Series的层次索引，其操作可参考df的loc操作-->data.loc[('A',slice(None)), :] 对于层次行索引，操作的是其内容 
- 可以通过unstack函数将数据倒排至一个df中(内层索引做为列 类似于数据透视的逆向过程)，其逆运算是stack函数    
- 设置多重索引，可以直接使用set_index函数  df.set_index(['class','id'])
- 多重索引创建：MultiIndex()、from_arrays()、from_product()
- df的层次行索引 数据获取最好使用loc或者ix函数，推荐使用loc函数；
   > **注意在对行索引的时候，若一级行索引还有多个，对二级行索引会遇到问题！也就是说，无法直接对二级索引进行索引，必须让二级索引变成一级索引后才能对其进行索引！** 可以使用swapllevel函数将内层索引的级别提高，然后获取，但是无法做出一二级行索引同时过滤获取数据； 最好将其unstack或者reindex之后在获取数据，这也是不建议使用层次化索引的原因；

In [63]:
## Series的层次化索引
data = pd.Series(np.random.randn(9),index=[['a', 'a', 'a', 'b', 'b', 'c', 'c','d', 'd'],[1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

data.index

# 1 使用部分索引选取子集(包括内层索引) 多种形式
data['b']
data['b':'c']
data.loc[['b','d']]  # data.ix[['b','d']]
data.loc[:,2] # 在内层获取 注意 此处的data为series，层次索引操作与df的操作一致

# 2 使用unstack倒排数据
data.unstack()

a  1    2.0796
   2    1.3963
   3   -1.3772
b  1    1.1744
   3   -1.1227
c  1    0.5295
   2   -0.8905
d  2    1.5393
   3    0.4531
dtype: float64

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

1    1.1744
3   -1.1227
dtype: float64

b  1    1.1744
   3   -1.1227
c  1    0.5295
   2   -0.8905
dtype: float64

b  1    1.1744
   3   -1.1227
d  2    1.5393
   3    0.4531
dtype: float64

a    1.3963
c   -0.8905
d    1.5393
dtype: float64

,1,2,3
a,2.0796,1.3963,-1.3772
b,1.1744,nan,-1.1227
c,0.5295,-0.8905,nan
d,nan,1.5393,0.4531


In [68]:
##  含有MultiIndex的DataFrame的操作
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
    index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],    
    columns=[['Ohio', 'Ohio', 'Colorado'],['Green', 'Red', 'Green']])

frame.index.names = ['key1','key2']


frame

# 1 根据部分索引选取数据
frame['Ohio'] # 最外层的列索引 根据列名获取
frame['Ohio']['Red'] # 获取内层列索引 根据列名获取 类似于多维数组的获取

frame.loc['b'] # 最外层的行索引 根据内容获取 最好使用loc属性获取 内部可以是list的形式的并列或者:形式


# frame.loc[:,'Ohio.Green']

# 2 构建MultiIndex来使用
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green','Red', 'Green']],names=['state', 'color'])

# 3 行索引优先级排序 
frame.swaplevel('key1','key2')  # 将key1、key2优先进行排序
frame.swaplevel(0,1) # 根据索引内容进行排序  同时交换级别
# frame.swaplevel(0,1).sortlevel(0)
# help(pd.DataFrame.swaplevel)

# 4 根据索引级别 使用参数level进行数据汇总
frame.sum(level='key2') # 行汇总
# frame.sum(level='color',axis=1)  # 列汇总 需指定axis=1 且 给出列名

Ohio     Colorado
          Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

key1  key2
a     1        1
      2        4
b     1        7
      2       10
Name: Red, dtype: int32

Ohio     Colorado
     Green Red    Green
key2                   
1        6   7        8
2        9  10       11

MultiIndex(levels=[['Colorado', 'Ohio'], ['Green', 'Red']],
           labels=[[1, 1, 0], [0, 1, 0]],
           names=['state', 'color'])

Ohio     Colorado
          Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

Ohio     Colorado
          Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

Ohio     Colorado
     Green Red    Green
key2                   
1        6   8       10
2       12  14       16

### 重排与分级排序
前面已经总结过，主要使用swaplevel函数，改变索引索引级别，也是获取内层行索引的一种方法，但还是无法根据一二级行索引同时过滤获取数据

In [69]:
frame.swaplevel('key1', 'key2')

frame.sort_index(level=1) # 根据level参数 使用内层索引

frame.swaplevel(0, 1).sort_index(level=0)

Ohio     Colorado
          Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

Ohio     Colorado
          Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

Ohio     Colorado
          Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### 根据级别汇总统计
主要使用的level参数进行级别指定，可适用于行列层次化索引

In [72]:
frame.sum(level='key2')

frame.columns.names = ['state','color']
frame.sum(level='color', axis=1)

Ohio     Colorado
     Green Red    Green
key2                   
1        6   8       10
2       12  14       16

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### 使用DataFrame的列进行索引
使用set_index设置索引生成单级或者层次索引，reset_index()/reindex()函数是将层次索引转化为默认索引

In [76]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
'c': ['one', 'one', 'one', 'two','two','two', 'two'],
'd': [0, 1, 2, 0, 1, 2, 3]})

frame.set_index(['c', 'd'], drop=False)
frame.reset_index(drop=True)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


## 合并数据集 
- 1、**左右合并 merge**      pd.merge(df1,df2,……)
> 1、如果不指定关联列，使用重叠列（列名相同的列）作为关联列  显示指定 on=    
> 2、无重叠列名，使用left_on 和 right_on 分别指定     
> 3、连接方式：inner（默认）、right、outer、left      
> 4、重复列名使用suffixes参数指定      

参数|说明
--|--
left|参与合并的左侧df
right|参与合并的右侧df
how| inner/outer/left/right 合并方式，默认inner
on| 连接的列名，前提是两个df具有该列，可以是一列col1，也可以是多列[col1,col2]
left_on | 指定左边df的连接列
right_on| 指定右边df的连接列
left_index | **True/False** 如果使用index进行连接，左边df的连接index（或者将index reset_index之后，进行列连接）
right_index| 类似left_index;若是层次索引就较为复杂 需要指定索引名称且left_index或者right_index设置为True
sort| 根据连接列进行排序，默认为True；禁用后可提升效率
suffixes| 重叠列名追加处理，也可连接后对colname进行重命名
copy| 默认为True，不改变原始数据

- 2、join **实现按索引合并** df1.join(df2,how='outer')  尽量不使用该函数  
> join函数使用场景较少，基本可以使用merge函数替代


- 3、concat 更广泛的df合并，所有的合并均可使用concat实现
> 不支持左右连接重的left/right join
> 可以沿不同的轴进行df合并，左右或者上下均可；
> 参数axis默认为0，实在y轴上进行连接，即union

参数|说明
--|--
objs| 参与连接的pandas对象的列表或者字典，唯一必须参数
axis| 指明连接的轴向 默认为0
join | inner/outer 默认是outer；指明其他轴向上的索引是按交集（inner）或者按并集（outer）进行合并
join_axis| 指明其他n-1条轴的索引，不执行并集交集运算
keys| 与连接对象相关的值，用于形成连接轴向上的层次化索引，可以是任意值的列表或者数组
levels| 指定用作层次化索引个各级别的索引，前提是设置了keys
names| 用于创建分层基本的名称，前提是设置了keys和levels
verify_integrity| 检查结果对象新轴上的重复情况
ignore_index| 不保留连接轴上的索引，产生一组新的索引 

具体参考[pandas 连接](https://www.cnblogs.com/HixiaoDi/p/7739863.html)

In [78]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a','b'],'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],'data2': range(3)})

df1
df2
                

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,key,data2
0,a,0
1,b,1
2,d,2


In [83]:
# 1 使用merge连接  如果不指定，根据相同的列名进行连接； 若指定，根据指定的列进行(on/left_on/right_on)
pd.merge(df1, df2, on='key')
# pd.merge(df1, df2, left_on='key', right_on='key')  

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [87]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'], 'key2': ['one', 'two', 'one'],'lval':[1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],'key2': ['one', 'one', 'one', 'two'],'rval': [4, 5, 6, 7]})
                     
left
right
pd.merge(left, right, on=['key1', 'key2'], how='outer')
pd.merge(left, right, on='key1', suffixes=('_left','_right'))

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


,key1,key2,lval,rval
0,foo,one,1.0000,4.0000
1,foo,one,1.0000,5.0000
2,foo,two,2.0000,nan
3,bar,one,3.0000,6.0000
4,bar,two,nan,7.0000


,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### 索引行的合并
就是使用索引进行df合并;对于层次化索引，是多键合并的索引，必须以列表的形式指明用作合并键的多个列

> DataFrame还有一个便捷的join实例方法，它能更为方便地实现按索引合并。
它还可用于合并多个**带有相同或相似索引的DataFrame对象**，但要求没有重叠的列。
该功能不常用，还是使用concat或者merge进行df合并

In [90]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'], 'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a','b'])

left1
right1

pd.merge(left1, right1, left_on='key', right_index=True)

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


,group_val
a,3.5000
b,7.0000


,key,value,group_val
0,a,0,3.5000
2,a,2,3.5000
3,a,3,3.5000
1,b,1,7.0000
4,b,4,7.0000


In [91]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
               'key2': [2000, 2001, 2002, 2001,2002],
               'data': np.arange(5.)})

righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
               index=[['Nevada', 'Nevada', 'Ohio','Ohio','Ohio', 'Ohio'],
                    [2001, 2000, 2000, 2000, 2001, 2002]],
               columns=['event1', 'event2'])

lefth
righth

# 注意：左表不是层次化索引
pd.merge(lefth, righth, left_on=['key1', 'key2'],right_index=True)  # 使用list形式声明做合并的多个列 

# 若两表都是层次化索引，使用left_index=True和right_index=True两个参数进行设置
# pd.merge(left2, right2, how='outer', left_index=True,right_index=True)

,key1,key2,data
0,Ohio,2000,0.0000
1,Ohio,2001,1.0000
2,Ohio,2002,2.0000
3,Nevada,2001,3.0000
4,Nevada,2002,4.0000


event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

### 轴向连接
- numpy数组可以使用np.concatenate函数进行
- pandas使用concat函数进行连接，默认方向是axis=0的连接，使用join参数指定连接方式（若需如此 直接使用merge连接）
- 至于层次化索引连接 以及 其他复杂操作，还是现将df转换后在进行连接  
- ignore_index参数 不保留原有索引，产生一组新索引

```python
help(pd.concat)
```

In [93]:
arr = np.arange(12).reshape((3, 4))
arr
np.concatenate([arr, arr], axis=1)  # axis=1 横向连接

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [94]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

pd.concat([s1, s2, s3])
pd.concat([s1, s2, s3], axis=1)

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [98]:
s4 = pd.concat([s1, s3])
pd.concat([s1, s4], axis=1, sort=False)
pd.concat([s1, s4], axis=1, join='inner') # 指定连接方式 
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b','e']]) # join_axes指定要在其它轴上使用的索引 

,0,1
a,0.0000,0
b,1.0000,1
f,nan,5
g,nan,6


,0,1
a,0,0
b,1,1


,0,1
a,0.0000,0.0000
c,nan,nan
b,1.0000,1.0000
e,nan,nan


### 合并重叠数据
当两个df的索引有部分或者部分重叠，使用combine_first方法，这个方法不常用~

In [101]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
              'b': [np.nan, 2., np.nan, 6.], 
              'c': range(2, 18, 4)})

df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
            'b': [np.nan, 3., 4., 6., 8.]})

df1
df2
df1.combine_first(df2) # 用传递对象中的数据为调用对象的缺失数据“打补丁” 用df2为df1打补丁

,a,b,c
0,1.0000,nan,2
1,nan,2.0000,6
2,5.0000,nan,10
3,nan,6.0000,14


,a,b
0,5.0000,nan
1,4.0000,3.0000
2,nan,4.0000
3,3.0000,6.0000
4,7.0000,8.0000


,a,b,c
0,1.0000,nan,2.0000
1,4.0000,2.0000,6.0000
2,5.0000,4.0000,10.0000
3,3.0000,6.0000,14.0000
4,7.0000,8.0000,nan


## 重塑和轴向旋转
主要是reshape和pivot系列函数
### 重塑层次化索引
- stack：将数据的列“旋转”为行
- unstack：将数据的行“旋转”为列。、


In [105]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio','Colorado'],name='state'),
                    columns=pd.Index(['one', 'two','three'],name='number'))
data
data.stack()  # 列转行 产生层次化索引
data.stack().unstack()

# 两者可以对不同级别的索引进行操作
result = data.stack()
result.unstack('state')

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [108]:
# 如果不是所有的级别值都能在各分组中找到的话，则unstack操作可能会引入缺失数据
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

data2.unstack()
data2.unstack().stack()
data2.unstack().stack(dropna=False)

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

,a,b,c,d,e
one,0.0000,1.0000,2.0000,3.0000,nan
two,nan,nan,4.0000,5.0000,6.0000


one  a   0.0000
     b   1.0000
     c   2.0000
     d   3.0000
two  c   4.0000
     d   5.0000
     e   6.0000
dtype: float64

one  a   0.0000
     b   1.0000
     c   2.0000
     d   3.0000
     e      nan
two  a      nan
     b      nan
     c   4.0000
     d   5.0000
     e   6.0000
dtype: float64

### 将长格式旋转为宽格式
> 将多列主键合并为单一主键 并进行数据透视pivot；pivot其实就是用set_index创建层次化索引，再用unstack重塑：

书中的列子有些复杂，主要是索引的构建。旋转、重构；一步一步跟着数据去理解

In [113]:
data = pd.read_csv('examples/macrodata.csv')
data.head()

periods = pd.PeriodIndex(year=data.year,quarter=data.quarter,name='date') # 构建时间序列字段 以季度为单位
columns = pd.Index(['realgdp', 'infl', 'unemp'],name='item') # 构建层次索引
data = data.reindex(columns=columns)  
data.index = periods.to_timestamp('D', 'end')  # 构建时间索引
ldata = data.stack().reset_index().rename(columns={0:'value'})
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0000,1.0000,2710.3490,1707.4000,286.8980,470.0450,1886.9000,28.9800,139.7000,2.8200,5.8000,177.1460,0.0000,0.0000
1,1959.0000,2.0000,2778.8010,1733.7000,310.8590,481.3010,1919.7000,29.1500,141.7000,3.0800,5.1000,177.8300,2.3400,0.7400
2,1959.0000,3.0000,2775.4880,1751.8000,289.2260,491.2600,1916.4000,29.3500,140.5000,3.8200,5.3000,178.6570,2.7400,1.0900
3,1959.0000,4.0000,2785.2040,1753.7000,299.3560,484.0520,1931.3000,29.3700,140.0000,4.3300,5.6000,179.3860,0.2700,4.0600
4,1960.0000,1.0000,2847.6990,1770.5000,331.7220,462.1990,1955.5000,29.5400,139.6000,3.5000,5.2000,180.0070,2.3100,1.1900


item,infl,realgdp,unemp
date,,,
1959-03-31,0.0000,2710.3490,5.8000
1959-06-30,2.3400,2778.8010,5.1000
1959-09-30,2.7400,2775.4880,5.3000
1959-12-31,0.2700,2785.2040,5.6000
1960-03-31,2.3100,2847.6990,5.2000
...,...,...,...
2008-09-30,-3.1600,13324.6000,6.0000
2008-12-31,-8.7900,13141.9200,6.9000
2009-03-31,0.9400,12925.4100,8.1000


### 将宽格式旋转为长格式
> 行转列,与stack类似，但是又不同；主要是非层次索引的快速行转列；

旋转DataFrame的逆运算是pandas.melt。它不是将一列转换到多个新的DataFrame，而是合并多个列成为一个，产生一个比输入长的DataFrame；


In [120]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
            'A': [1, 2, 3],
            'B': [4, 5, 6],
            'C': [7, 8, 9]})

df
melted = pd.melt(df, ['key'])
melted

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [125]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped
reshaped.reset_index()

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [127]:
# 指定列的子集，作为值的列
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

# 不用分组指标
pd.melt(df, value_vars=['A', 'B', 'C'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9
